In [ ]:
"""
process_model.py

Load COBRA models from an input folder, close exchange reactions for metabolites
whose formula contains 'C' not followed by a lowercase letter, and save modified
models to an output folder.
"""

import os
import cobra


def process_model(model):
    """
    Process the given COBRA model by setting exchange reactions to zero
    for metabolites with a 'C' in their formula not followed by a lowercase letter.

    Parameters:
        model (cobra.Model): The COBRA model to process.
    """
    medium = model.medium  # Get the current medium

    # Iterate through all metabolites in the medium
    for metabolite_id, concentration in medium.items():
        # Get the metabolite object by removing the 'EX_' prefix
        metabolite = model.metabolites.get_by_id(metabolite_id.replace('EX_', ''))

        # Check if the formula contains 'C' not followed by a lowercase letter
        if 'C' in metabolite.formula:
            idx = metabolite.formula.index('C')
            next_char = metabolite.formula[idx + 1] if idx + 1 < len(metabolite.formula) else None

            # Ensure the next character is not a lowercase letter
            if next_char is None or not next_char.islower():
                print(f"Closing exchange reaction for {metabolite_id} (Formula: {metabolite.formula})")
                medium[metabolite_id] = 0  # Set this exchange reaction to zero

    # Update the model's medium
    model.medium = medium


def main(input_folder, output_folder):
    """
    Load models from the input folder, process them, and save the modified
    models to the output folder.

    Parameters:
        input_folder (str): Path to the folder containing XML model files.
        output_folder (str): Path to the folder where modified models will be saved.
    """
    # Ensure the output folder exists
    os.makedirs(output_folder, exist_ok=True)

    # Iterate over all XML files in the input folder
    for file_name in os.listdir(input_folder):
        if file_name.endswith('.xml'):
            file_path = os.path.join(input_folder, file_name)

            try:
                # Load the model
                model = cobra.io.read_sbml_model(file_path)

                # Process the model
                process_model(model)

                # Save the modified model to the output folder
                output_file_path = os.path.join(output_folder, file_name)
                cobra.io.write_sbml_model(model, output_file_path)
                print(f"Processed and saved: {output_file_path}")

            except Exception as e:
                print(f"Error processing {file_name}: {e}")


if __name__ == "__main__":
    # Set the input and output folder paths
    input_folder = 'E:/model/mqc_control'      # Replace with your input folder path
    output_folder = 'path/to/output/models'    # Replace with your output folder path
    main(input_folder, output_folder)